In [1]:
import sys
sys.path.append("..")

In [2]:
from utils import DataGenerator, read_annotation_lines
from models import Yolov4
from config import yolo_config

In [ ]:
import os
from tensorflow import keras 

dir4saving = '../checkpoints'
os.makedirs(dir4saving, exist_ok = False)

logdir = '../logs'
os.makedirs(logdir, exist_ok = False)

name4saving = 'epoch_{epoch:02d}-val_loss-{val_loss:.4f}.hdf5'

filepath = os.path.join(dir4saving, name4saving)

rLrCallBack = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                             factor = 0.1,
                                             patience = 5,
                                             verbose = 1)

tbCallBack = keras.callbacks.TensorBoard(log_dir = logdir,
                                         histogram_freq = 0,
                                         write_graph = False,
                                         write_images = False)

mcCallBack_loss = keras.callbacks.ModelCheckpoint(filepath,
                                            monitor = 'val_loss',
                                            verbose = 1,
                                            save_best_only = True,
                                            save_weights_only = False,
                                            mode = 'auto',
                                            period = 1)

esCallBack = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                          mode = 'min',
                                          verbose = 1,
                                          patience = 10)


In [ ]:
train_lines, val_lines = read_annotation_lines('../dataset/txt/anno-test.txt', test_size=0.1)

FOLDER_PATH = '../dataset/img'
class_name_path = '../class_names/coco_classes.txt'

data_gen_train = DataGenerator(train_lines, class_name_path, FOLDER_PATH)
data_gen_val = DataGenerator(val_lines, class_name_path, FOLDER_PATH)

model = Yolov4(weight_path=None, 
               class_name_path=class_name_path)

model.fit(data_gen_train, 
          initial_epoch=0,
          epochs=10000, 
          val_data_gen=data_gen_val,
          callbacks=[])

model.save_model('../')